In [1]:
from elasticsearch import Elasticsearch, helpers
import regex as re
from subprocess import Popen, PIPE, STDOUT
import os
import pandas as pd
import re

from tqdm.notebook import tqdm

import numpy as np

tqdm.pandas(desc="Progress")


In [2]:
# Connect to ES instance
es_server = Elasticsearch(
    hosts=["http://localhost:9200"], timeout=60, retry_on_timeout=True)


<ipython-input-2-640b3a38b6a4>:2: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es_server = Elasticsearch(


## GeoNames

In [3]:
df_geonames = pd.read_csv('../allCountries.txt', sep='\t')
df_geonames.columns = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 
                       'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code', 'admin2_code', 'admin3_code', 'admin4_code',
                       'population', 'elevation', 'dem', 'timezone', 'modification_date']
df_geonames

/home/breno/miniconda3/envs/insightlab/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
0,2994701,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74028,T,MT,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2803,Europe/Andorra,2020-06-10
1,3007683,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,PK,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2685,Europe/Andorra,2014-11-05
2,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD,FR,A9,66,663.0,66146.0,0,NaN,2411,Europe/Andorra,2014-11-05
3,3017833,Estany de les Abelletes,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.52915,1.73362,H,LK,AD,FR,A9,NaN,NaN,NaN,0,NaN,2260,Europe/Andorra,2014-11-05
4,3023203,Port Vieux de la Coume d’Ose,Port Vieux de la Coume d'Ose,"Port Vieux de Coume d'Ose,Port Vieux de Coume ...",42.62568,1.61823,T,PASS,AD,NaN,00,NaN,NaN,NaN,0,NaN,2687,Europe/Andorra,2014-11-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12081471,12217934,Baltic States,Baltic States,https://en.wikipedia.org/wiki/Baltic_states,56.85514,25.32515,L,RGN,NaN,"EE,LT,LV",NaN,NaN,NaN,NaN,6040000,NaN,111,Europe/Riga,2021-03-14
12081472,12218088,Arab Gulf Countries,Arab Gulf Countries,"Arab states of the Persian Gulf,Arabian Gulf S...",26.35058,50.16805,L,RGN,NaN,"AE,BH,IQ,KW,OM,QA,SA",NaN,NaN,NaN,NaN,95080000,NaN,12,Asia/Riyadh,2021-03-24
12081473,12218168,South China Sea,South China Sea,NaN,14.64735,115.63737,H,SEA,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-9999,NaN,2021-03-17
12081474,12225504,Polar Regions,Polar Regions,NaN,85.00000,50.00000,L,RGN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-9999,NaN,2021-04-06


In [4]:
df_geonames[df_geonames['geonameid'] == 3469034]

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
792682,3469034,Federative Republic of Brazil,Federative Republic of Brazil,"An Bhrasail,An Bhrasaíl,Ba Tay (Bra-xin),Ba Tâ...",-10.0,-55.0,A,PCLI,BR,NaN,0,NaN,NaN,NaN,209469333,NaN,258,NaN,2020-03-29


In [5]:
mapping = {
    "mappings": {
        "properties": {'geonameid': {'type': 'long', "index": False}, 
                       'name': {'type': 'text'}, 'asciiname': {'type': 'text'}, 'alternatenames': {'type':'text'},
                       'name2': {'type': 'keyword'}, 'asciiname2': {'type': 'keyword'}, 'alternatenames2': {'type':'keyword'},
                       'latitude': {'type':'double', "index": False}, 'longitude':{'type':'double', "index": False}, 'location':{'type':'geo_point', "index": False},
                       'feature_class': {'type':'text', "index": False}, 'feature_code': {'type':'text', "index": False},
                       'country_code': {'type':'text'}, 'cc2': {'type':'text', "index": False},
                       'admin1_code': {'type':'text', "index": False}, 'admin2_code': {'type':'text', "index": False},
                       'admin3_code': {'type':'text', "index": False}, 'admin4_code': {'type':'text', "index": False},
                       'population': {'type':'long', "index": False}, 'population2': {'type':'long', "index": False}, 'elevation': {'type':'text', "index": False},
                       'dem': {'type':'long', "index": False}, 
                       'timezone': {'type':'text'}, 'modification_date': {'type':'date', "index": False}}
    }
}


# make an API call to the Elasticsearch cluster
# and have it return a response:
response = es_server.indices.create(
    index="geonames",
    body=mapping,
    ignore=400
)
response

<ipython-input-5-f090488e5ed8>:20: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es_server.indices.create(
<ipython-input-5-f090488e5ed8>:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_server.indices.create(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geonames'})

In [6]:
p_map = {"PPLC": 100000, "PCLI": 100000, "PCL": 100000, "PCLS": 10000, "PCLF": 10000, "CONT": 100000, "RGN": 100000}

def populate_index(entries, index="geonames"):
  es_buffer = []
  rows = 0
  for entry in entries:
    entry["_id"] = entry["geonameid"]
    entry["_index"] = index

    if "alternatenames" in entry:
      entry["alternatenames"] = entry.get("alternatenames").split(',')  

    entry["name2"] = entry.get("name", "").lower()
    entry["asciiname2"] = entry.get("asciiname", "").lower()
    entry["alternatenames2"] = [x.lower() for x in entry.get("alternatenames", [])]

    entry["population2"] = entry.get("population", 0)
    if entry["population2"] == 0 and entry.get("feature_class") in ["A", "P", "L"]:
        entry["population2"] = p_map.get(entry.get("feature_code"), 0)

    entry["location"] = f"{entry['latitude']}, {entry['longitude']}"

    es_buffer.append(entry)

    rows += 1
    if rows % 1000 == 0:
      helpers.bulk(es_server, es_buffer)
      es_buffer = []

  if es_buffer:
    helpers.bulk(es_server, es_buffer)

  print("Total entries inserted: {}".format(rows), end="\r")

In [7]:
batch_size = 5000

for batch_number, batch_df in tqdm(df_geonames.groupby(np.arange(len(df_geonames)) // batch_size)):
    populate_index([ {k:v for k,v in m.items() if pd.notnull(v)} for m in batch_df.to_dict(orient='records')], index="geonames")
'''
for batch_number, batch_df in tqdm(df_geonames[df_geonames['country_code'] == 'US'].groupby(np.arange(len(df_geonames[df_geonames['country_code'] == 'US'])) // batch_size)):
    populate_index([ {k:v for k,v in m.items() if pd.notnull(v)} for m in batch_df.to_dict(orient='rows')], index="geonames")
'''

batch_df = None

  0%|          | 0/2417 [00:00<?, ?it/s]

## Adjetivos

In [3]:
mapping = {
    "mappings": {
        "properties": {
                       'entity_name': {'type': 'text'}, 'entity_name_lower': {'type': 'keyword'},
                       'geonameid': {'type': 'long', "index": False}, 
                       'geonames_name': {'type': 'text'}, 'geonames_name_lower': {'type': 'keyword'},
                       'adjectivals': {'type':'text'}, 'adjectivals_lower': {'type':'keyword'},
                       'demonyms': {'type':'text'}, 'demonyms_lower': {'type':'keyword'},
        }
    }
}


# make an API call to the Elasticsearch cluster
# and have it return a response:
response = es_server.indices.create(
    index="demonyms",
    body=mapping,
    ignore=400
)
response

<ipython-input-3-997e8ecdd65b>:16: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es_server.indices.create(
<ipython-input-3-997e8ecdd65b>:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_server.indices.create(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'demonyms'})

In [4]:
df_demonyms = pd.read_csv('../demonyms.csv')
df_demonyms

,Entity Name,Adjectival,Demonym,Geonames Id,Geonames Name
0,Abkhazia,"Abkhaz, Abkhazian",Abkhazians,6643410,Abkhazia
1,Afghanistan,Afghan,Afghans,1149361,Islamic Republic of Afghanistan
2,Åland Islands,Åland Island,Åland Islanders,661882,Ahvenanmaan Lääni
3,Albania,Albanian,Albanians,783754,Republic of Albania
4,Algeria,Algerian,Algerians,2589581,People’s Democratic Republic of Algeria
...,...,...,...,...,...
259,Western Sahara,"Sahrawi, Sahrawian, Sahraouian","Sahrawis, Sahraouis",2461445,Western Sahara
260,Yemen,Yemeni,Yemenis,69543,Republic of Yemen
261,Zambia,Zambian,Zambians,895949,Republic of Zambia
262,Zanzibar,Zanzibari,Zanzibaris,148726,Zanzibar Island


In [5]:
def populate_demonyms(entries, index="demonyms"):
  es_buffer = []
  rows = 0
  for entry in entries:
    es_entry = {}
    es_entry["_index"] = index

    es_entry["geonames_id"] = entry["Geonames Id"]
    es_entry["_id"] = es_entry["geonames_id"]

    es_entry["geonames_name"] = entry.get("Geonames Name", "")
    es_entry["geonames_name_lower"] = es_entry.get("geonames_name", "").lower()
    
    es_entry["entity_name"] = entry.get("Entity Name", "")
    es_entry["entity_name_lower"] = es_entry.get("entity_name", "").lower()

    es_entry["adjectivals"] = entry.get("Adjectival", "").split(',')
    es_entry["adjectivals_lower"] = [x.lower() for x in es_entry.get("adjectivals", [])]

    es_entry["demonyms"] = entry.get("Demonym", "").split(',')
    es_entry["demonyms_lower"] = [x.lower() for x in es_entry.get("demonyms", [])]

    es_buffer.append(es_entry)

    rows += 1
    if rows % 1000 == 0:
      helpers.bulk(es_server, es_buffer)
      es_buffer = []

  if es_buffer:
    helpers.bulk(es_server, es_buffer)

  print("Total entries inserted: {}".format(rows), end="\r")

In [6]:
batch_size = 5000

for batch_number, batch_df in tqdm(df_demonyms.groupby(np.arange(len(df_demonyms)) // batch_size)):
    populate_demonyms([ {k:v for k,v in m.items() if pd.notnull(v)} for m in batch_df.to_dict(orient='records')], index="demonyms")

batch_df = None

  0%|          | 0/1 [00:00<?, ?it/s]

## Snapshot

In [12]:
!curl -X PUT "http://localhost:9200/_snapshot/es_repo?pretty" -H 'Content-Type: application/json' -d '{"type": "fs", "settings": { "location": "/usr/share/elasticsearch/es_repo",  "compress": true} }'

{
  "acknowledged" : true
}


In [13]:
es_server.snapshot.get_repository(name="es_repo")

ObjectApiResponse({'es_repo': {'type': 'fs', 'settings': {'compress': 'true', 'location': '/usr/share/elasticsearch/es_repo'}}})

In [14]:
!curl -XPUT "http://localhost:9200/_snapshot/es_repo/geonames_snapshot_demonyms?wait_for_completion=true" -H 'Content-Type: application/json' -d '{ "ignore_unavailable": true, "include_global_state": false }'

{"snapshot":{"snapshot":"geonames_snapshot_demonyms","uuid":"9pZbbLLYR96j8SrlwO9EMg","repository":"es_repo","version_id":7140299,"version":"7.14.2","indices":["demonyms",".geoip_databases","geonames"],"data_streams":[],"include_global_state":false,"state":"SUCCESS","start_time":"2022-11-11T06:35:24.082Z","start_time_in_millis":1668148524082,"end_time":"2022-11-11T06:36:49.141Z","end_time_in_millis":1668148609141,"duration_in_millis":85059,"failures":[],"shards":{"total":3,"failed":0,"successful":3},"feature_states":[]}}

In [15]:
es_server.snapshot.status(repository='es_repo', snapshot='geonames_snapshot_demonyms')

ObjectApiResponse({'snapshots': [{'snapshot': 'geonames_snapshot_demonyms', 'repository': 'es_repo', 'uuid': '9pZbbLLYR96j8SrlwO9EMg', 'state': 'SUCCESS', 'include_global_state': False, 'shards_stats': {'initializing': 0, 'started': 0, 'finalizing': 0, 'done': 3, 'failed': 0, 'total': 3}, 'stats': {'incremental': {'file_count': 275, 'size_in_bytes': 3048742169}, 'total': {'file_count': 275, 'size_in_bytes': 3048742169}, 'start_time_in_millis': 1668148524082, 'time_in_millis': 85059}, 'indices': {'.geoip_databases': {'shards_stats': {'initializing': 0, 'started': 0, 'finalizing': 0, 'done': 1, 'failed': 0, 'total': 1}, 'stats': {'incremental': {'file_count': 22, 'size_in_bytes': 41231582}, 'total': {'file_count': 22, 'size_in_bytes': 41231582}, 'start_time_in_millis': 1668148524082, 'time_in_millis': 1200}, 'shards': {'0': {'stage': 'DONE', 'stats': {'incremental': {'file_count': 22, 'size_in_bytes': 41231582}, 'total': {'file_count': 22, 'size_in_bytes': 41231582}, 'start_time_in_milli

In [16]:
es_server.snapshot.get(repository = 'es_repo', snapshot = '_all')

ObjectApiResponse({'snapshots': [{'snapshot': 'geonames_snapshot_demonyms', 'uuid': '9pZbbLLYR96j8SrlwO9EMg', 'repository': 'es_repo', 'version_id': 7140299, 'version': '7.14.2', 'indices': ['demonyms', '.geoip_databases', 'geonames'], 'data_streams': [], 'include_global_state': False, 'state': 'SUCCESS', 'start_time': '2022-11-11T06:35:24.082Z', 'start_time_in_millis': 1668148524082, 'end_time': '2022-11-11T06:36:49.141Z', 'end_time_in_millis': 1668148609141, 'duration_in_millis': 85059, 'failures': [], 'shards': {'total': 3, 'failed': 0, 'successful': 3}, 'feature_states': []}]})

## Teste

In [17]:
def get_coordinates_es(loc_name, exact_matching=True):
    txt = loc_name.lower()
    if exact_matching: 
      query = {
        "query": {
          "bool": {
            "should": [
              { "term": { "name2": txt }},
              { "term": { "asciiname2": txt }},
              { "term": { "alternatenames2": txt}}
            ]
          }
        }
      }
    else:
      query = {
        "query": {
          "bool": {
            "must": {
              "multi_match": {
                "query": txt,
                "fields": ['name', 'asciiname', 'alternatenames'],
                "operator":   "and"
              }
            }
          }
        }
      }
    result = []
    for query_result in es_server.search(index="geonames", body=query, size=50)["hits"]["hits"]:
        source = query_result["_source"]
        result.append((source["latitude"], source["longitude"], source.get("population", 0), source.get("feature_code", "W"), source.get("alternatenames2")))
    result.sort(key=lambda candidate: candidate[2], reverse=True)
    return result

In [18]:
get_coordinates_es("Brazil")

<ipython-input-17-fc4681e2cfdf>:30: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  for query_result in es_server.search(index="geonames", body=query, size=50)["hits"]["hits"]:


[(-10.0,
  -55.0,
  209469333,
  'PCLI',
  ['an bhrasail',
   'an bhrasaíl',
   'ba tay (bra-xin)',
   'ba tây (bra-xin)',
   'baraasiil',
   'beresiil',
   'berezili',
   'birazil',
   'bra-xin',
   'braasiil',
   'brasiilia',
   'brasil',
   'brasila',
   'brasilaen',
   'brasile',
   'brasili',
   'brasilia',
   'brasilie',
   'brasilien',
   'brasilië',
   'brasilän',
   'brasilía',
   'brasíle',
   'brasīl',
   'braxil',
   'braziel',
   'braziilii',
   'brazil',
   'brazil nutome',
   'brazili',
   'brazilia',
   'brazilie',
   'brazilija',
   'braziliya',
   'brazilië',
   'brazilo',
   'brazylia',
   'brazylje',
   'brazíl',
   'brazília',
   'brazílie',
   'brazīlija',
   'brażil',
   'breasail',
   'bresi',
   'bresil',
   'brezil',
   'brezila',
   'brezili',
   'brezilya',
   'brezílɛ',
   'brezîli',
   'brèsil',
   'brési',
   'brésil',
   'buraziiri',
   'burezili',
   'estados unidos do brasil',
   'estados unidos do brazil',
   'federative republic of brazil',
   'mnule

In [19]:
def get_full_name_es(demonym, exact_matching=True):
    txt = demonym.lower()
    if exact_matching: 
      query = {
        "query": {
          "bool": {
            "should": [
              { "term": { "adjectivals_lower": txt }},
              { "term": { "demonyms_lower": txt }}
            ]
          }
        }
      }
    else:
      query = {
        "query": {
          "bool": {
            "must": {
              "multi_match": {
                "query": txt,
                "fields": ['adjectivals', 'demonyms'],
                "operator":   "and"
              }
            }
          }
        }
      }
    result = []
    for query_result in es_server.search(index="demonyms", body=query, size=50)["hits"]["hits"]:
        source = query_result["_source"]
        result.append((source["entity_name"], source["geonames_id"], source["geonames_name"], source["adjectivals"], source["demonyms"]))
    return result

In [20]:
get_full_name_es("Sahrawi", False)

<ipython-input-19-f4d483a0e729>:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  for query_result in es_server.search(index="demonyms", body=query, size=50)["hits"]["hits"]:


[('Western Sahara',
  2461445,
  'Western Sahara',
  ['Sahrawi', ' Sahrawian', ' Sahraouian'],
  ['Sahrawis', ' Sahraouis'])]